## Radiotoxicity chart tool
### Inventory-based Spent Fuel Simulation

### 1. Intialization

In [ ]:
# Imports
import sys
import importlib
import numpy as np
from pathlib import Path

#Project directories
sys.path.append(str(Path().resolve() / "Backend" / "Inventory-based Spent Fuel Simulation"))
import find_project_root_inv
importlib.reload(find_project_root_inv)
from find_project_root_inv import find_project_root_inv
project_root = find_project_root_inv()
backend_path = project_root / "Backend"
main_output_dir = project_root / "Simulation results"

In [ ]:
# Resource file Paths
excel_file_fuel = project_root / "Resources" / "SpentFuel_Data.xlsx"
excel_file_DC = project_root / "Resources" / "Dose coefficients.xlsx"
chain_file = project_root / "Resources" / "chain_endfb71_pwr.xml"

In [ ]:
# Reactor selection
selected_sheet = "B.7 - UO2"

# DC selection
selected_DC = "Adults"

### 2. Simulation of Spent Fuel Inventory Decay (only needed if RT data not already existing)

In [ ]:
# Time steps
time_steps = np.concatenate((
    np.logspace(0, 3, num=80),   # 1 to nearly 1000
    np.logspace(3, 6, num=40)    # from 1000 to 1e6
))

In [ ]:
# Run Simulaiton
from simulation_inv import simulation_inv
simulation_inv(project_root, main_output_dir, excel_file_fuel, time_steps, selected_sheet, selected_DC, chain_file)

### 3. Select Data Path (Plot Input Data/ RT Calculation Input Data and Output Path)

In [ ]:
# Show file chooser
from file_chooser_inv import file_chooser_inv
chooser = file_chooser_inv(main_output_dir)

In [ ]:
# Debug selected path
print("Selected path:", chooser.selected_path)

### 4. Radiotoxicity calculation (only needed if RT data not already existing)

In [ ]:
# Run RT calculation
from RT_calc_inv import RT_calc_inv
RT_results = RT_calc_inv(chooser, excel_file_DC, selected_DC, excel_file_fuel, selected_sheet)

### 5. Plotting

### 5.1 Plot configuration

In [ ]:
save_plot = False

In [ ]:
# Cooling time/Data offset
offset = 0

In [ ]:
# Plot settings
plot_config = {

    #Graph linestyle
    "linewidth" : 2.0,

    #Figure size
    "fig_width" : 12,
    "fig_height" : 7,

    #Axis
    "plot_axis_range_x" : [1e0, 1e6],
    "plot_axis_range_y" : [1e0, 1e11],

    "xticks_fontsize" : 12,
    "yticks_fontsize" : 12,

    #Titles
    "plot_title" : f"Radiotoxicity of Spent Fuel (inventory-based)", #no slashes allowed

    "plot_subtitle" : "SF at discharge, UO2, 4.5%, 65 GWd/t, Adults",

    "plot_xlabel" : "Time (years)",
    "plot_ylabel" : "Radiotoxicity [Sv/t$_{IHM}$]",

    #Footnote
    "show_footnote" : True,
    "footnote_text" : "IHM = Initial Heavy Metal\nSF = Spent Fuel",

    #Font name and sizes
    "font_name" : 'DejaVu Sans',
    "title_fontsize" : 14,
    "label_fontsize" : 13,
    "legend_fontsize" : 12,
    "subtitle_fontsize" : 12,

    #Legend location
    "show_legend" : True,
    "plot_legend_loc" : 'upper right',
    "legend_outside" : False,  #False = inside 

    #Grid
    "plot_grid_show" : True,
    "plot_grid_which" : 'major',
    "plot_grid_linestyle" : '--',
    "plot_grid_linewidth" : 0.5,

    #resolution
    "dpi" : 300,
}

In [ ]:
# Show UI
from plot_UI_inv import plot_UI_inv
checkboxes, color_pickers, isotope_textboxes, isotope_colors = plot_UI_inv()

In [ ]:
# Reference value configuration
ref_values = {
    "ref_value1": 256000,    #Ref value for 8.41t of natural U in equlibrium with its daughter products (ingestion/adults)
    #"ref_value2": ...
}
    
ref_config = {
    "ref_value1": {"label": "Natural Uranium (8.41t)", "linestyle": "--", "linewidth": 2, "color": "#ee0000"},
    #"ref_value2": ...
    
    "legend_bbox_anchor": (1.287, 1),  #horizontal position of legend if outside
}

### 5.2 Plot chart

In [ ]:
# Plot
from plot_inv import plot_inv
decay_isotopes, all_isotopes_in_RT, group_isotopes, fission_isotopes = plot_inv(project_root, chooser, checkboxes, color_pickers, isotope_textboxes, isotope_colors, excel_file_fuel, selected_sheet, selected_DC, save_plot, offset, ref_values, ref_config, plot_config)

#### Show concentrations

In [ ]:
from debug_read_conc import debug_read_conc
debug_read_conc(chooser)

#### Show radiotoxicities

In [ ]:
def format_float_list(values, precision=4):
    return ", ".join(f"{v:.{precision}e}" for v in values)

print("\nRT values computed:")
for nuclide, rt in RT_results.items():
    print(f"{nuclide}: [{format_float_list(rt)}]")

#### Show Isotopes considered Decay Products

In [ ]:
decay_isotopes = list(all_isotopes_in_RT - group_isotopes - set(fission_isotopes))

print("Isotopes considered Decay Products")
print(f"Count: {len(decay_isotopes)}")
print("Isotopes: " + ", ".join(decay_isotopes))
